# Análisis archivo de cargas SAP

In [5]:
import pandas as pd
import datetime
import os
from tkinter import filedialog

username = os.getlogin()

hoy = datetime.datetime.now()
date_format = "%d.%m.%Y"
date_formatexport = "%d-%m-%Y"
date_formatcartola = "%d/%m/%Y"
date_formatcarga = "%Y%m%d"
fecha_actual = hoy.strftime(date_format)
fecha_cartola = hoy.strftime(date_formatexport)

folder_path = 'C:\\Users\\' + username + '\\Documents\\Py\\CARGASSAP\\ANALISIS'

print("Cargando archivos...")
df_list = []
files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
for file in files:
    df = pd.read_csv(os.path.join(folder_path, file),
                        sep=';',header=None,
                        engine='python',
                        #dtype='int64'
                        )
    df_list.append(df)
df_cargas = pd.concat(df_list)
df_cargas = df_cargas.dropna(axis=1)
                       
df_cargas.columns = ['flowid','counter','account','period','post','constant1','sa','gl','monto','indcont','user','ref','agencia','doc']

df_parc = pd.read_csv('C:\\Users\\' + username + '\\Documents\\Py\\CARGASSAP\\PARCIALIZADOS\\conteoparcializados.csv',
                        sep=',',
                        names=['agencia','count'],
                        header='infer')

#df_cg['monto'] = df['monto'].astype(int)

#df_ident = pd.merge(df, df_agencias, how='left', left_on='monto', right_on='monto')
df_cargas['agencia'] = df_cargas['agencia'].replace('AG. ','',regex=True)
df_cargas['agencia'] = df_cargas['agencia'].str.slice(0,-3)

df_adosc = df_cargas[df_cargas['monto'] == 200000]
count_adosc = df_adosc.groupby('agencia').size().reset_index(name='count').sort_values(by='count',ascending=False)
df_acien = df_cargas[df_cargas['monto'] == 100000]
count_acien = df_acien.groupby('agencia').size().reset_index(name='count').sort_values(by='count',ascending=False)

df_depositos = pd.merge(count_adosc,count_acien,on='agencia',how='outer')
df_depositos.columns = ['agencia','depdosc','depcien']
df_depositos = pd.merge(df_depositos,df_parc,on='agencia',how='left')
df_depositos['depdosc'] = df_depositos['depdosc'].fillna('0').astype(int)
df_depositos['depcien'] = df_depositos['depcien'].fillna('0').astype(int)
df_depositos['count'] = df_depositos['count'].fillna('0').astype(int)

Cargando archivos...


In [17]:
df1.to_csv('C:\\Users\\' + username + '\\Documents\\Py\\DEVOLTX\\Reporte CARGAS' + fecha_actual + '.csv',
            sep=';',
            header=False,
            index=False)

In [5]:
tipos_fecha = df1['fecha'].unique()
tipos_sucursal = df1['sucursal'].unique()
tipos_operacion = df1['operacion'].unique()
tipos_descripcion = df1['descripcion'].unique()